In [1]:
!pip install requests

In [2]:
import requests
import datetime
from urllib.parse import urlencode  #bringing this in so that we can properly create urls needed for the search api below

In [3]:
import base64

In [4]:
# (1) need a spotify account (2) go to the developer spotify website (3) dashboard page has the client_id and client_secret, referenced throughout this Jupyter notebook

client_id = '......'         #<----------------- YOUR ID HERE
    
client_secret = '.....'       #<----------------- YOUR SECRET HERE


In [9]:
# do a lookup for a token -- which will be used for future requests, it's like authenticating to a session
# for this project I will be defining the spotifyapi

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.client_id = client_id
            self.client_secret = client_secret
    
    def get_client_credentials(self):
            """
            Returns a base 64 encoded string
            """
            client_id = self.client_id
            client_secret = self.client_secret
            if client_secret == None or client_id == None:
                raise Exception("You must set client_id and client_secret")
            client_creds = f"{client_id}:{client_secret}"
            client_creds_b64 = base64.b64encode(client_creds.encode())
            return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return  {
                "Authorization": f"Basic {client_creds_b64}" 
        }
        
    def get_token_data(self):
        return {
                "grant_type": "client_credentials"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range (200, 299):
            raise Exception("Could not authenticate client.")
            # return False   ---- commenting out because we probably want to raise an exception instead of just saying false
        data = r.json()
        now = datetime.datetime.now() # gives the exact of now, whenever something is requested
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in) # will give a datetime object relative to now, based off when the request is happening 
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        auth_done = self.perform_auth()
        if not auth_done:
            raise Exception("Authentication failed")
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization" : f"Bearer {access_token}"
        }
        return headers
    
    def get_resources(self, lookup_id, resource_type='shows', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return{}
        return r.json()
    
    def get_show (self, _id):
        return self.get_resources(_id, resource_type='show')
    
    def get_episode(self, _id):
        return self.get_resources(_id, resource_type='episode')
    
    def base_search (self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"        
        lookup_url = f"{endpoint}?{query_params}"    
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='show'):                     #creating another here to have a more robust query search
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)


In [10]:
spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
# Help here please! I'm trying to find data for my favorite podcast Code Switch .. it exists and is popular so there should definitely be data here ... https://open.spotify.com/show/3bExJ9JQpkwNhoHvaIIuyV

In [14]:
spotify.search({"show":"Code Switch", "episode":"Claim Us If You're Famous"}, search_type="episode")

q=show%3ACode+Switch+episode%3AClaim+Us+If+You%27re+Famous&type=episode


{'episodes': {'href': 'https://api.spotify.com/v1/search?query=show%3ACode+Switch+episode%3AClaim+Us+If+You%27re+Famous&type=episode&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [ ]:
#spotify.search("Code Switch", search_type="show")
#spotify.search("Black And Up In Arms", search_type="episode")

In [15]:
spotify.search({'id':"3bExJ9JQpkwNhoHvaIIuyV"}, search_type="episode")

q=id%3A3bExJ9JQpkwNhoHvaIIuyV&type=episode


{'episodes': {'href': 'https://api.spotify.com/v1/search?query=id%3A3bExJ9JQpkwNhoHvaIIuyV&type=episode&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}